# importing packages

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm

np.random.seed(2022)

# relhum conversion functions

In [2]:
def eliq(T):
    a_liq = np.float32(np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]));
    c_liq = np.float32(-80.0)
    T0 = np.float32(273.16)
    return np.float32(100.0)*np.polyval(a_liq,np.maximum(c_liq,T-T0))

# def eiceOriginal(T):
#     a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
#                       0.602588177e-7,0.615021634e-5,0.420895665e-3,\
#                       0.188439774e-1,0.503160820,6.11147274]));
#     c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
#     T0 = np.float32(273.16)
#     return np.transpose(np.where(T>c_ice[0],eliq(T),\
#                    np.transpose(np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
#                    (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
#                            np.float32(100.0)*np.polyval(a_ice,T-T0)))))

def eice(T):
    a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]));
    c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
    T0 = np.float32(273.16)
    return np.where(T>c_ice[0],eliq(T),\
                   np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
                           np.float32(100.0)*np.polyval(a_ice,T-T0)))

# def esatOriginal(T):
#     T0 = np.float32(273.16)
#     T00 = np.float32(253.16)
#     omtmp = (T-T00)/(T0-T00)
#     omega = np.maximum(np.float32(0.0),np.minimum(np.float32(1.0),omtmp))
#     return np.transpose(np.where(T>T0,eliq(T),np.transpose(np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))))

def esat(T):
    T0 = np.float32(273.16)
    T00 = np.float32(253.16)
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(np.float32(0.0),np.minimum(np.float32(1.0),omtmp))
    return np.where(T>T0,eliq(T),np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

# def qv(T,RH,P0,PS,hyam,hybm):

#     R = np.float32(287.0)
#     Rv = np.float32(461.0)
#     p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)

#     T = np.float32(T)
#     RH = np.float32(RH)
#     p = np.float32(p)

#     return R*esat(T)*RH/(Rv*p)
    # DEBUG 1
    # return esat(T)

def RH(T,qv,P0,PS,hyam,hybm):
    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    T = np.float32(T)
    qv = np.float32(qv)
    p = np.float32(p)
    
    return Rv*p*qv/(R*esat(T))

# data processing functions

In [8]:
def doMonth(month):
    datasets = !ls
    n = str(month)
    datasets = [x for x in datasets if "h1.0000-" + n.zfill(2) in x]
    return xr.open_mfdataset(datasets)

def makeSuffix(month):
    n = str(month)
    return "_" + n.zfill(2)

def saveNNInput(month):
    spData = doMonth(month)
    suffix = makeSuffix(month)
    print("read in data")
    nntbp = spData["NNTBP"].values
    nnqbp = spData["NNQBP"].values
    p0 = spData["P0"].values
    ps = spData["NNPS"].values
    hyam = spData["hyam"].values
    hybm = spData["hybm"].values
    relhum = spData["RELHUM"].values
    tphystnd = spData["TPHYSTND"].values
    phq = spData["PHQ"].values

    p0 = np.array(list(set(p0)))
    print("loaded in data")
    newhum = np.zeros((spData["time"].shape[0],\
                         spData["lev"].shape[0], \
                         spData["lat"].shape[0], \
                         spData["lon"].shape[0]))
    lats = spData["lat"]
    lons = spData["lon"]
    print("starting for loop")
    for i in tqdm(range(len(lats))):
        for j in range(len(lons)):
            latIndex = i
            lonIndex = j
            R = np.float32(287.0)
            Rv = np.float32(461.0)
            p = p0 * hyam + ps[:, None, latIndex, lonIndex] * hybm # Total pressure (Pa)

            T = np.float32(nntbp[:, :, latIndex, lonIndex])
            qv = np.float32(nnqbp[:, :, latIndex, lonIndex])
            p = np.float32(p)
            newhum[:,:, latIndex, lonIndex] = Rv*p*qv/(R*esat(T))
    
    nntbp = np.moveaxis(nntbp[1:,:,:,:],0,1)
    print("nntbp")
    print(nntbp.shape)
    
    nnqbp = np.moveaxis(nnqbp[1:,:,:,:],0,1)
    print("nnqbp")
    print(nnqbp.shape)
    
    lhflx = spData["LHFLX"].values[np.newaxis,:-1,:,:]
    print("lhflx")
    print(lhflx.shape)
    
    shflx = spData["SHFLX"].values[np.newaxis,:-1,:,:]
    print("shflx")
    print(shflx.shape)
    
    ps = spData["NNPS"].values[np.newaxis,1:,:,:]
    print("ps")
    print(ps.shape)
    
    solin = spData["SOLIN"].values[np.newaxis,1:,:,:]
    print("solin")
    print(solin.shape)
    
    newhum = np.moveaxis(newhum[1:,:,:,:],0,1)
    print("newhum")
    print(newhum.shape)
    
    oldhum = np.moveaxis(relhum[1:,:,:,:],0,1)
    print("oldhum")
    print(oldhum.shape)
    
    tphystnd = np.moveaxis(tphystnd[1:,:,:,:],0,1)
    print("tphystnd")
    print(tphystnd.shape)
    

    phq = np.moveaxis(phq[1:,:,:,:],0,1)
    print("phq")
    print(phq.shape)

    nnInput = np.concatenate((nntbp, \
                              nnqbp, \
                              lhflx, \
                              shflx, \
                              ps, \
                              solin, \
                              newhum, \
                              oldhum, \
                              tphystnd, \
                              phq))
    print("nnInput")
    nnInput.shape

    errors = (newhum-oldhum/100).flatten()
    result = "Mean error: " + str(np.mean(errors)) + "\n"
    result = result + "Variance: " + str(np.var(errors)) + "\n"
    result = result + "nntbp.shape: " + str(nntbp.shape) + "\n"
    result = result + "nnqbp.shape: " + str(nnqbp.shape) + "\n"
    result = result + "lhflx.shape: " + str(lhflx.shape) + "\n"
    result = result + "shflx.shape: " + str(shflx.shape) + "\n"
    result = result + "ps.shape: " + str(ps.shape) + "\n"
    result = result + "solin.shape: " + str(solin.shape) + "\n"
    result = result + "newhum.shape: " + str(newhum.shape) + "\n"
    result = result + "oldhum.shape: " + str(oldhum.shape) + "\n"
    result = result + "tphystnd.shape: " + str(tphystnd.shape) + "\n"
    result = result + "phq.shape: " + str(phq.shape) + "\n"
    result = result + "nnInput.shape: " + str(nnInput.shape) + "\n"
    print(result)
    
    #added 32 bit fix
    nnInput = np.float32(nnInput)
    
    fileName = 'nnInput' + suffix + '.npy'
    with open(fileName, 'wb') as f:
        np.save(f, nnInput)

    diagnostics = 'diagnostics' + suffix + '.txt'
    with open(diagnostics, 'a') as fp:
        fp.write(result)

# run the code

In [4]:
#saveNNInput(1)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [04:13<00:00,  3.96s/it]


nntbp
(30, 1007, 64, 128)
nnqbp
(30, 1007, 64, 128)
lhflx
(1, 1007, 64, 128)
shflx
(1, 1007, 64, 128)
ps
(1, 1007, 64, 128)
solin
(1, 1007, 64, 128)
newhum
(30, 1007, 64, 128)
oldhum
(30, 1007, 64, 128)
tphystnd
(30, 1007, 64, 128)
phq
(30, 1007, 64, 128)
nnInput
Mean error: 0.00449642062535382
Variance: 0.000531489944120111
nntbp.shape: (30, 1007, 64, 128)
nnqbp.shape: (30, 1007, 64, 128)
lhflx.shape: (1, 1007, 64, 128)
shflx.shape: (1, 1007, 64, 128)
ps.shape: (1, 1007, 64, 128)
solin.shape: (1, 1007, 64, 128)
newhum.shape: (30, 1007, 64, 128)
oldhum.shape: (30, 1007, 64, 128)
tphystnd.shape: (30, 1007, 64, 128)
phq.shape: (30, 1007, 64, 128)
nnInput.shape: (184, 1007, 64, 128)



In [5]:
#saveNNInput(2)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [05:38<00:00,  5.28s/it]


nntbp
(30, 1343, 64, 128)
nnqbp
(30, 1343, 64, 128)
lhflx
(1, 1343, 64, 128)
shflx
(1, 1343, 64, 128)
ps
(1, 1343, 64, 128)
solin
(1, 1343, 64, 128)
newhum
(30, 1343, 64, 128)
oldhum
(30, 1343, 64, 128)
tphystnd
(30, 1343, 64, 128)
phq
(30, 1343, 64, 128)
nnInput
Mean error: 0.0045253398222425325
Variance: 0.0005452406919383343
nntbp.shape: (30, 1343, 64, 128)
nnqbp.shape: (30, 1343, 64, 128)
lhflx.shape: (1, 1343, 64, 128)
shflx.shape: (1, 1343, 64, 128)
ps.shape: (1, 1343, 64, 128)
solin.shape: (1, 1343, 64, 128)
newhum.shape: (30, 1343, 64, 128)
oldhum.shape: (30, 1343, 64, 128)
tphystnd.shape: (30, 1343, 64, 128)
phq.shape: (30, 1343, 64, 128)
nnInput.shape: (184, 1343, 64, 128)



In [6]:
#saveNNInput(3)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [06:13<00:00,  5.83s/it]


nntbp
(30, 1487, 64, 128)
nnqbp
(30, 1487, 64, 128)
lhflx
(1, 1487, 64, 128)
shflx
(1, 1487, 64, 128)
ps
(1, 1487, 64, 128)
solin
(1, 1487, 64, 128)
newhum
(30, 1487, 64, 128)
oldhum
(30, 1487, 64, 128)
tphystnd
(30, 1487, 64, 128)
phq
(30, 1487, 64, 128)
nnInput
Mean error: 0.0047049172968508315
Variance: 0.0006586717710367489
nntbp.shape: (30, 1487, 64, 128)
nnqbp.shape: (30, 1487, 64, 128)
lhflx.shape: (1, 1487, 64, 128)
shflx.shape: (1, 1487, 64, 128)
ps.shape: (1, 1487, 64, 128)
solin.shape: (1, 1487, 64, 128)
newhum.shape: (30, 1487, 64, 128)
oldhum.shape: (30, 1487, 64, 128)
tphystnd.shape: (30, 1487, 64, 128)
phq.shape: (30, 1487, 64, 128)
nnInput.shape: (184, 1487, 64, 128)



In [7]:
#saveNNInput(4)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [06:01<00:00,  5.65s/it]


nntbp
(30, 1439, 64, 128)
nnqbp
(30, 1439, 64, 128)
lhflx
(1, 1439, 64, 128)
shflx
(1, 1439, 64, 128)
ps
(1, 1439, 64, 128)
solin
(1, 1439, 64, 128)
newhum
(30, 1439, 64, 128)
oldhum
(30, 1439, 64, 128)
tphystnd
(30, 1439, 64, 128)
phq
(30, 1439, 64, 128)
nnInput
Mean error: 0.004646388751251582
Variance: 0.0006922037939262922
nntbp.shape: (30, 1439, 64, 128)
nnqbp.shape: (30, 1439, 64, 128)
lhflx.shape: (1, 1439, 64, 128)
shflx.shape: (1, 1439, 64, 128)
ps.shape: (1, 1439, 64, 128)
solin.shape: (1, 1439, 64, 128)
newhum.shape: (30, 1439, 64, 128)
oldhum.shape: (30, 1439, 64, 128)
tphystnd.shape: (30, 1439, 64, 128)
phq.shape: (30, 1439, 64, 128)
nnInput.shape: (184, 1439, 64, 128)



In [8]:
#saveNNInput(5)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [06:13<00:00,  5.84s/it]


nntbp
(30, 1487, 64, 128)
nnqbp
(30, 1487, 64, 128)
lhflx
(1, 1487, 64, 128)
shflx
(1, 1487, 64, 128)
ps
(1, 1487, 64, 128)
solin
(1, 1487, 64, 128)
newhum
(30, 1487, 64, 128)
oldhum
(30, 1487, 64, 128)
tphystnd
(30, 1487, 64, 128)
phq
(30, 1487, 64, 128)
nnInput
Mean error: 0.004552564777648614
Variance: 0.0006317513355610316
nntbp.shape: (30, 1487, 64, 128)
nnqbp.shape: (30, 1487, 64, 128)
lhflx.shape: (1, 1487, 64, 128)
shflx.shape: (1, 1487, 64, 128)
ps.shape: (1, 1487, 64, 128)
solin.shape: (1, 1487, 64, 128)
newhum.shape: (30, 1487, 64, 128)
oldhum.shape: (30, 1487, 64, 128)
tphystnd.shape: (30, 1487, 64, 128)
phq.shape: (30, 1487, 64, 128)
nnInput.shape: (184, 1487, 64, 128)



In [9]:
#saveNNInput(6)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [06:01<00:00,  5.65s/it]


nntbp
(30, 1439, 64, 128)
nnqbp
(30, 1439, 64, 128)
lhflx
(1, 1439, 64, 128)
shflx
(1, 1439, 64, 128)
ps
(1, 1439, 64, 128)
solin
(1, 1439, 64, 128)
newhum
(30, 1439, 64, 128)
oldhum
(30, 1439, 64, 128)
tphystnd
(30, 1439, 64, 128)
phq
(30, 1439, 64, 128)
nnInput
Mean error: 0.004568632873742671
Variance: 0.0005829908510743547
nntbp.shape: (30, 1439, 64, 128)
nnqbp.shape: (30, 1439, 64, 128)
lhflx.shape: (1, 1439, 64, 128)
shflx.shape: (1, 1439, 64, 128)
ps.shape: (1, 1439, 64, 128)
solin.shape: (1, 1439, 64, 128)
newhum.shape: (30, 1439, 64, 128)
oldhum.shape: (30, 1439, 64, 128)
tphystnd.shape: (30, 1439, 64, 128)
phq.shape: (30, 1439, 64, 128)
nnInput.shape: (184, 1439, 64, 128)



In [10]:
#saveNNInput(7)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [06:12<00:00,  5.81s/it]


nntbp
(30, 1487, 64, 128)
nnqbp
(30, 1487, 64, 128)
lhflx
(1, 1487, 64, 128)
shflx
(1, 1487, 64, 128)
ps
(1, 1487, 64, 128)
solin
(1, 1487, 64, 128)
newhum
(30, 1487, 64, 128)
oldhum
(30, 1487, 64, 128)
tphystnd
(30, 1487, 64, 128)
phq
(30, 1487, 64, 128)
nnInput
Mean error: 0.004624294810867391
Variance: 0.0006135179197601439
nntbp.shape: (30, 1487, 64, 128)
nnqbp.shape: (30, 1487, 64, 128)
lhflx.shape: (1, 1487, 64, 128)
shflx.shape: (1, 1487, 64, 128)
ps.shape: (1, 1487, 64, 128)
solin.shape: (1, 1487, 64, 128)
newhum.shape: (30, 1487, 64, 128)
oldhum.shape: (30, 1487, 64, 128)
tphystnd.shape: (30, 1487, 64, 128)
phq.shape: (30, 1487, 64, 128)
nnInput.shape: (184, 1487, 64, 128)



In [11]:
#saveNNInput(8)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [06:14<00:00,  5.85s/it]


nntbp
(30, 1487, 64, 128)
nnqbp
(30, 1487, 64, 128)
lhflx
(1, 1487, 64, 128)
shflx
(1, 1487, 64, 128)
ps
(1, 1487, 64, 128)
solin
(1, 1487, 64, 128)
newhum
(30, 1487, 64, 128)
oldhum
(30, 1487, 64, 128)
tphystnd
(30, 1487, 64, 128)
phq
(30, 1487, 64, 128)
nnInput
Mean error: 0.004418762130331022
Variance: 0.0005444266637220918
nntbp.shape: (30, 1487, 64, 128)
nnqbp.shape: (30, 1487, 64, 128)
lhflx.shape: (1, 1487, 64, 128)
shflx.shape: (1, 1487, 64, 128)
ps.shape: (1, 1487, 64, 128)
solin.shape: (1, 1487, 64, 128)
newhum.shape: (30, 1487, 64, 128)
oldhum.shape: (30, 1487, 64, 128)
tphystnd.shape: (30, 1487, 64, 128)
phq.shape: (30, 1487, 64, 128)
nnInput.shape: (184, 1487, 64, 128)



In [12]:
#saveNNInput(9)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [06:01<00:00,  5.66s/it]


nntbp
(30, 1439, 64, 128)
nnqbp
(30, 1439, 64, 128)
lhflx
(1, 1439, 64, 128)
shflx
(1, 1439, 64, 128)
ps
(1, 1439, 64, 128)
solin
(1, 1439, 64, 128)
newhum
(30, 1439, 64, 128)
oldhum
(30, 1439, 64, 128)
tphystnd
(30, 1439, 64, 128)
phq
(30, 1439, 64, 128)
nnInput
Mean error: 0.004478826859563261
Variance: 0.0005605209436831268
nntbp.shape: (30, 1439, 64, 128)
nnqbp.shape: (30, 1439, 64, 128)
lhflx.shape: (1, 1439, 64, 128)
shflx.shape: (1, 1439, 64, 128)
ps.shape: (1, 1439, 64, 128)
solin.shape: (1, 1439, 64, 128)
newhum.shape: (30, 1439, 64, 128)
oldhum.shape: (30, 1439, 64, 128)
tphystnd.shape: (30, 1439, 64, 128)
phq.shape: (30, 1439, 64, 128)
nnInput.shape: (184, 1439, 64, 128)



In [13]:
#saveNNInput(10)

100%|██████████████████████████████████████████████████████████████████████| 64/64 [06:14<00:00,  5.85s/it]


nntbp
(30, 1487, 64, 128)
nnqbp
(30, 1487, 64, 128)
lhflx
(1, 1487, 64, 128)
shflx
(1, 1487, 64, 128)
ps
(1, 1487, 64, 128)
solin
(1, 1487, 64, 128)
newhum
(30, 1487, 64, 128)
oldhum
(30, 1487, 64, 128)
tphystnd
(30, 1487, 64, 128)
phq
(30, 1487, 64, 128)
nnInput
Mean error: 0.004394172820393663
Variance: 0.0005414692008619802
nntbp.shape: (30, 1487, 64, 128)
nnqbp.shape: (30, 1487, 64, 128)
lhflx.shape: (1, 1487, 64, 128)
shflx.shape: (1, 1487, 64, 128)
ps.shape: (1, 1487, 64, 128)
solin.shape: (1, 1487, 64, 128)
newhum.shape: (30, 1487, 64, 128)
oldhum.shape: (30, 1487, 64, 128)
tphystnd.shape: (30, 1487, 64, 128)
phq.shape: (30, 1487, 64, 128)
nnInput.shape: (184, 1487, 64, 128)



In [7]:
saveNNInput(11)

read in data
loaded in data
starting for loop


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [06:00<00:00,  5.63s/it]


nntbp
(30, 1439, 64, 128)
nnqbp
(30, 1439, 64, 128)
lhflx
(1, 1439, 64, 128)
shflx
(1, 1439, 64, 128)
ps
(1, 1439, 64, 128)
solin
(1, 1439, 64, 128)
newhum
(30, 1439, 64, 128)
oldhum
(30, 1439, 64, 128)
tphystnd
(30, 1439, 64, 128)
phq
(30, 1439, 64, 128)
nnInput
Mean error: 0.004063744879412078
Variance: 0.00044028795292243045
nntbp.shape: (30, 1439, 64, 128)
nnqbp.shape: (30, 1439, 64, 128)
lhflx.shape: (1, 1439, 64, 128)
shflx.shape: (1, 1439, 64, 128)
ps.shape: (1, 1439, 64, 128)
solin.shape: (1, 1439, 64, 128)
newhum.shape: (30, 1439, 64, 128)
oldhum.shape: (30, 1439, 64, 128)
tphystnd.shape: (30, 1439, 64, 128)
phq.shape: (30, 1439, 64, 128)
nnInput.shape: (184, 1439, 64, 128)



In [8]:
saveNNInput(12)

read in data
loaded in data
starting for loop


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [06:15<00:00,  5.86s/it]


nntbp
(30, 1487, 64, 128)
nnqbp
(30, 1487, 64, 128)
lhflx
(1, 1487, 64, 128)
shflx
(1, 1487, 64, 128)
ps
(1, 1487, 64, 128)
solin
(1, 1487, 64, 128)
newhum
(30, 1487, 64, 128)
oldhum
(30, 1487, 64, 128)
tphystnd
(30, 1487, 64, 128)
phq
(30, 1487, 64, 128)
nnInput
Mean error: 0.004069206737723481
Variance: 0.0004647211701387928
nntbp.shape: (30, 1487, 64, 128)
nnqbp.shape: (30, 1487, 64, 128)
lhflx.shape: (1, 1487, 64, 128)
shflx.shape: (1, 1487, 64, 128)
ps.shape: (1, 1487, 64, 128)
solin.shape: (1, 1487, 64, 128)
newhum.shape: (30, 1487, 64, 128)
oldhum.shape: (30, 1487, 64, 128)
tphystnd.shape: (30, 1487, 64, 128)
phq.shape: (30, 1487, 64, 128)
nnInput.shape: (184, 1487, 64, 128)

